In [18]:
!apt-get update && apt-get install -y default-libmysqlclient-dev build-essential

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.5ubuntu2).
The following additional packages will be installed:
  cpp cpp-9 dpkg-dev fakeroot g++ g++-9 gcc gcc-9 libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libdpkg-perl libfakeroot
  libfile-fcntllock-perl libgcc-9-dev liblocale-gettext-perl libstdc++-9-dev
  make xz-utils
Suggested packages:
  cpp-doc gcc-9-locales debian-keyring g++-multilib g++-9-multilib gcc-9-doc
  gcc-multilib manpages-dev libtool flex bison gdb gcc-doc gcc-9-mul

In [19]:
!pip install mysqlclient

  Using cached mysqlclient-2.1.1.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp39-cp39-linux_x86_64.whl size=60515 sha256=c7f3d91613706baf7b18f519fc81a9b9b52f8b71cedbac525a28f7ea9603a8aa
  Stored in directory: /root/.cache/pip/wheels/f3/a5/27/c6312d8008951cfd5511684378a9e057b82006c70e1fea6107
Successfully built mysqlclient


In [25]:
from dask_kubernetes.experimental import KubeCluster

cluster = KubeCluster(name="rapids-dask",
                     image="rapidsai/rapidsai-core:22.08-cuda11.5-runtime-ubuntu20.04-py3.9",
                     worker_command="dask-cuda-worker",
                     n_workers=2,
                     resources={"limits": {"nvidia.com/gpu": "1"}},
                     env={"DISABLE_JUPYTER": "true"})

In [26]:
cluster

AttributeError: 'Tab' object has no attribute '_ipython_display_'

KubeCluster(rapids-dask, 'tcp://rapids-dask-cluster-service.kubeflow-user-example-com:8786', workers=2, threads=2, memory=29.30 GiB)

In [10]:
import optuna
import joblib
import dask.distributed

In [11]:
def objective(trial):
    x = trial.suggest_uniform("x", -10, 10)
    return (x - 2) ** 2

In [34]:
with dask.distributed.Client(cluster) as client:
    study = optuna.create_study(storage="mysql://root@10.36.2.20:3306/metadb")
    # Optimize in parallel on your Dask cluster
    with joblib.parallel_backend("dask"):
        study.optimize(objective, n_trials=100, n_jobs=-1)
    print(f"best_params = {study.best_params}")

[I 2022-09-26 17:29:27,449] A new study created in RDB with name: no-name-ad98d40a-e408-406b-8483-5ec7b17f88b1
/tmp/ipykernel_96/638899847.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  x = trial.suggest_uniform("x", -10, 10)
[I 2022-09-26 17:29:27,632] Trial 0 finished with value: 16.52593701625623 and parameters: {'x': 6.0652105746512355}. Best is trial 0 with value: 16.52593701625623.
[I 2022-09-26 17:29:27,676] Trial 3 finished with value: 2.873495827170803 and parameters: {'x': 0.3048611186186534}. Best is trial 3 with value: 2.873495827170803.
[I 2022-09-26 17:29:27,682] Trial 1 finished with value: 128.04925668261677 and parameters: {'x': -9.31588514799513}. Best is trial 3 with value: 2.873495827170803.
[I 2022-09-26 17:29:27,696] Trial 2 finished with value: 32.75744160573581 and parameters: {'x': -3.723

best_params = {'x': 1.9933550847224837}


In [35]:
study.best_params

{'x': 1.9933550847224837}

In [36]:
study.best_value

4.415489904536975e-05